# Disambiguating Place Names analysis

- **people**: BIRTH_PLACE, DEATH_PLACE, OCCUPATION
- **objects**: PLACE_MADE, MATERIALS (via Getty)

In [16]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")
import os

from heritageconnector.config import config
from heritageconnector.utils.data_transformation import transform_series_str_to_list
from heritageconnector.entity_matching.reconciler import reconciler

from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
df = pd.read_pickle("../../GITIGNORE_DATA/results/filtering_people_orgs_result.pkl")
df_people = df[df['GENDER'].isin(["M", "F"])]

## 1. Automated

Place names doesn't work because the idea of a 'place of birth' is too wide.

In [18]:
df_people['OCCUPATION'] = transform_series_str_to_list(df_people['OCCUPATION'], separator=";")

df_people.head(1)

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered
1,10245,"Zenthon, Edward Rupert",NaN,Edward Rupert,Zenthon,NaN,NaN,M,Y,REF: http://www.iwm.org.uk/collections/item/object/1030031461,nan,1920-07,"London, Greater London, England, United Kingdom",c. 2002,NaN,NaN,British,[engineer],NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,05-AUG-15,REF: http://www.iwm.org.uk/collections/item/object/1030031461 --- nan,[],[http://www.iwm.org.uk/collections/item/object/1030031461],[]


In [57]:
df_people["BIRTH_PLACE"] = df_people["BIRTH_PLACE"].fillna("")
testdf = df_people.head(100)
rec = reconciler(testdf, table="PERSON")

#df_people["OCCUPATION_qids"] = rec.process_column("OCCUPATION", multiple_vals=True)
rec.process_column("BIRTH_PLACE", multiple_vals=False)


/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
  0%|          | 0/26 [00:00<?, ?it/s]

Looking up Wikidata qcodes for unique items..


100%|██████████| 26/26 [00:13<00:00,  1.97it/s]


1      []
2      []
3      []
4      []
5      []
       ..
201    []
204    []
209    []
211    []
214    []
Name: BIRTH_PLACE, Length: 100, dtype: object

In [58]:
lookup_filter = reconciler.get_subject_items_from_pid("P19")
lookup_filter

['Q958822', 'Q1322263']

# 2. Manual

**Result:** look ups aren't working because the *instanceof_filter* defined using the PID isn't correct. When trying to use a high-level filter (see below), the results are too varied.

**Next step:** this is an NLP problem: geocoding. We can frame it as an entity linking problem instead.

In [28]:
from heritageconnector.disambiguation.search import wikidata_text_search

In [29]:
search = wikidata_text_search()

In [33]:
search.run_search("Germany", instanceof_filter=["Q2221906"], include_class_tree=True)

,rank,item,itemLabel,score
0,1,http://www.wikidata.org/entity/Q1350565,Germany,0.142857
1,2,http://www.wikidata.org/entity/Q1011486,Burg Drachenfels,0.131868
2,3,http://www.wikidata.org/entity/Q49289431,Setzingen,0.120879
3,4,http://www.wikidata.org/entity/Q2268218,Geilenkirchen-Heinsberg,0.109890
4,5,http://www.wikidata.org/entity/Q55134129,Germany,0.098901
5,6,http://www.wikidata.org/entity/Q49292935,Werda,0.087912
6,7,http://www.wikidata.org/entity/Q49292059,Undenheim,0.076923
7,8,http://www.wikidata.org/entity/Q16855868,Pädagogische Hochschule Erfurt/Mühlhausen,0.065934
8,9,http://www.wikidata.org/entity/Q7033,Nordhausen,0.054945
9,10,http://www.wikidata.org/entity/Q183,Germany,0.043956


### Manual labelling

24% of birth places can also be obtained through manual labelling of only the 100 top place names. ~50% of the collection doesn't have a birth place so this is equivalent to 50%.

In [62]:
n_labelled = 100
(df_people.loc[df_people['BIRTH_PLACE'] != "", 'BIRTH_PLACE'].value_counts() / len(df_people) * 100)[0:n_labelled].sum()

24.082302936630605